In [1]:
import os
import sys
import requests
from PIL import Image
from PIL.ExifTags import TAGS
from SPARQLWrapper import SPARQLWrapper, JSON

# Définition du point de terminaison Wikidata SPARQL
endpoint_url = "https://query.wikidata.org/sparql"

# Requête SPARQL pour récupérer les informations sur les tableaux
query = """
SELECT DISTINCT ?painting ?paintingLabel ?artistLabel ?genreLabel ?locationLabel ?date ?width ?height ?image WHERE {
  ?painting wdt:P31 wd:Q3305213.  # Sélectionne les peintures
  OPTIONAL { ?painting wdt:P170 ?artist. }  # Artiste
  OPTIONAL { ?painting wdt:P136 ?genre. }  # Genre artistique
  OPTIONAL { ?painting wdt:P276 ?location. }  # Lieu de conservation (musée)
  OPTIONAL { ?painting wdt:P571 ?date. }  # Date de création
  OPTIONAL { ?painting wdt:P2049 ?width. } # Largeur du tableau
  OPTIONAL { ?painting wdt:P2048 ?height. } # Hauteur du tableau
  OPTIONAL { ?painting wdt:P18 ?image. }  # Image du tableau

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr,en". }
}
LIMIT 100
"""

# Fonction pour exécuter la requête SPARQL
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Fonction pour récupérer les dimensions EXIF d'une image
def get_exif_data(image_url):
    try:
        # Télécharger l'image
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        # Ouvrir l'image avec PIL
        image = Image.open(response.raw)

        # Récupérer les dimensions
        width, height = image.size

        # Récupérer les métadonnées EXIF
        exif_data = image._getexif()
        exif_info = {}
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                exif_info[tag_name] = value

        # Déterminer l'orientation
        orientation = "Paysage (Horizontal)" if width > height else "Portrait (Vertical)"

        return {
            "ImageWidth": width,
            "ImageHeight": height,
            "Orientation": orientation,
            "EXIF_Metadata": exif_info
        }

    except Exception as e:
        return {"error": str(e)}

# Récupération des données Wikidata
array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    painting_name = result.get("paintingLabel", {}).get("value", "N/A")
    artist = result.get("artistLabel", {}).get("value", "N/A")
    genre = result.get("genreLabel", {}).get("value", "N/A")
    location = result.get("locationLabel", {}).get("value", "N/A")
    date = result.get("date", {}).get("value", "N/A")
    width = result.get("width", {}).get("value", "N/A")
    height = result.get("height", {}).get("value", "N/A")
    image_url = result.get("image", {}).get("value", "N/A")

    # Récupérer les métadonnées EXIF si une image est disponible
    exif_data = get_exif_data(image_url) if image_url != "N/A" else {}

    array.append(
        (
            painting_name,  # Nom du tableau
            artist,  # Nom de l'artiste
            genre,  # Genre artistique
            location,  # Lieu de conservation
            date,  # Date de création
            width,  # Largeur (Wikidata)
            height,  # Hauteur (Wikidata)
            image_url,  # URL de l'image
            exif_data.get("ImageWidth", "N/A"),  # Largeur EXIF
            exif_data.get("ImageHeight", "N/A"),  # Hauteur EXIF
            exif_data.get("Orientation", "N/A"),  # Orientation EXIF
            exif_data.get("EXIF_Metadata", {}),  # Métadonnées EXIF complètes
        )
    )

# Affichage des résultats
for item in array:
    print(item)


('Séjour dans les Monts Fuchun', 'Huang Gongwang', 'paysage', 'N/A', '1348-01-01T00:00:00Z', 'N/A', 'N/A', 'http://commons.wikimedia.org/wiki/Special:FilePath/%E5%AF%8C%E6%98%A5%E5%B1%B1%E5%B1%85%E5%9C%96%28%E7%84%A1%E7%94%A8%E5%B8%AB%E5%8D%B7%29.jpg', 'N/A', 'N/A', 'N/A', {})
('La Laitière', 'Johannes Vermeer', 'scène de genre', 'Gallery of Honour', '1660-01-01T00:00:00Z', '41', '45.5', 'http://commons.wikimedia.org/wiki/Special:FilePath/Johannes%20Vermeer%20-%20Het%20melkmeisje%20-%20Google%20Art%20Project.jpg', 'N/A', 'N/A', 'N/A', {})
('La Madone Solly', 'Raphaël', 'art sacré', 'Gemäldegalerie', '1501-01-01T00:00:00Z', '38', '52', 'http://commons.wikimedia.org/wiki/Special:FilePath/1502%20Raffael%20Madonna%20Solly%20anagoria.JPG', 'N/A', 'N/A', 'N/A', {})
('La Mort de la Vierge', 'Le Caravage', 'art sacré', 'salle 712', '1604-01-01T00:00:00Z', '245', '369', 'http://commons.wikimedia.org/wiki/Special:FilePath/Death%20of%20the%20Virgin-Caravaggio%20%281606%29.jpg', 'N/A', 'N/A', 'N/A

In [4]:
import pandas as pd
import json  # Pour stocker les métadonnées EXIF en JSON

# Définition des colonnes du DataFrame
columns = [
    "tableau", "artiste", "genre", "lieu", "date",
    "largeur_wikidata", "hauteur_wikidata",
    "image", "largeur_exif", "hauteur_exif", "orientation_exif", "exif_metadata"
]

# Transformation des métadonnées EXIF en JSON (évite les erreurs avec Pandas)
array_cleaned = []
for item in array:
    cleaned_item = list(item)
    cleaned_item[-1] = json.dumps(cleaned_item[-1])  # Convertir le dictionnaire en JSON
    array_cleaned.append(cleaned_item)

# Création du DataFrame
dataframe = pd.DataFrame(array_cleaned, columns=columns)

# Correction des types pour éviter les erreurs
dataframe = dataframe.astype(str)

# Affichage du DataFrame
dataframe


,tableau,artiste,genre,lieu,date,largeur_wikidata,hauteur_wikidata,image,largeur_exif,hauteur_exif,orientation_exif,exif_metadata
0,Séjour dans les Monts Fuchun,Huang Gongwang,paysage,N/A,1348-01-01T00:00:00Z,N/A,N/A,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
1,La Laitière,Johannes Vermeer,scène de genre,Gallery of Honour,1660-01-01T00:00:00Z,41,45.5,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
2,La Madone Solly,Raphaël,art sacré,Gemäldegalerie,1501-01-01T00:00:00Z,38,52,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
3,La Mort de la Vierge,Le Caravage,art sacré,salle 712,1604-01-01T00:00:00Z,245,369,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
4,Manaò tupapaú,Paul Gauguin,portrait,Galerie d'Art Albright-Knox,1892-01-01T00:00:00Z,92.4,72.4,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Portrait de Paul III avec ses petits-fils,Titien,portrait de famille,musée de Capodimonte,1546-01-01T00:00:00Z,176,210,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
96,Nocturne en bleu et or - le Vieux Pont de Batt...,James Abbott McNeill Whistler,paysage,Tate Britain,1872-01-01T00:00:00Z,68.3,51.2,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
97,Rêve causé par le vol d'une abeille autour d'u...,Salvador Dalí,surréalisme,musée Thyssen-Bornemisza,1944-01-01T00:00:00Z,40.5,51,N/A,N/A,N/A,N/A,{}
98,Le Moine au bord de la mer,Caspar David Friedrich,paysage,Alte Nationalgalerie,1800-01-01T00:00:00Z,171.5,110,http://commons.wikimedia.org/wiki/Special:File...,N/A,N/A,N/A,{}
